In [1]:
%pip install --upgrade --quiet  langchain-core langchain-community langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


In [4]:
OPENAI_API_KEY = ""

In [5]:
prompt = ChatPromptTemplate.from_template("tell me short joke about {topic}")
model = ChatOpenAI(openai_api_key = OPENAI_API_KEY)
output_parser = StrOutputParser()
chain = prompt | model | output_parser

In [6]:
chain.invoke({"topic":"icecream"})

'Why did the ice cream truck break down? Because it had too many "scoops"!'

In [7]:
prompt = ChatPromptTemplate.from_template("tell me short joke about {topic}")
model = ChatOpenAI(openai_api_key = OPENAI_API_KEY)
# output_parser = StrOutputParser()
chain = prompt | model 

In [8]:
chain.invoke({"topic":"icecream"})

AIMessage(content='Why did the ice cream go to therapy? Because it had too many sprinkles of anxiety!', response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 14, 'total_tokens': 33}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})

In [9]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

In [12]:
!pip install langchain docarray tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.2/270.2 kB 4.8 MB/s eta 0:00:00a 0:00:01


In [15]:
vectorstore =  DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"], embedding = OpenAIEmbeddings(openai_api_key = OPENAI_API_KEY)
)
retriver = vectorstore.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(openai_api_key = OPENAI_API_KEY)
output_parser = StrOutputParser()
setup_and_retrival = RunnableParallel(
    {"context":retriver,"question":RunnablePassthrough()}
)
chain  = setup_and_retrival | prompt | model | output_parser

In [16]:
chain.invoke("harrison worked at ?")

'Kensho'

In [17]:
chain.invoke("harrison likes to eat ?")

'We cannot answer the question based on the provided context.'

In [18]:
#####################

In [19]:
%pip install --upgrade --quiet  langchain-core langchain-openai langchain-anthropic

Note: you may need to restart the kernel to use updated packages.


In [20]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
output_parser = StrOutputParser()
model = ChatOpenAI(openai_api_key = OPENAI_API_KEY)
chain = (
    {"topic": RunnablePassthrough()} 
    | prompt
    | model
    | output_parser
)

chain.invoke("ice cream")

"Why did the ice cream break up with the spoon? Because it couldn't handle the rocky road!"

In [22]:
#Stream

In [21]:
for chunk in chain.stream("ice cream"):
    print(chunk, end="", flush=True)

Why did the ice cream truck break down?
Because it had too many "scoops" of ice cream!

In [23]:
#Batch

In [24]:
chain.batch(["ice cream", "spaghetti", "dumplings"])

['Why did the ice cream truck break down? \n\nBecause it had too many "scoops" of ice cream!',
 'Why did the spaghetti go to the party? Because it heard it was going to be a pasta-tively great time!',
 'Why did the dumpling go to the party? Because it wanted to get steamed!']